Import the required library:

In [2]:
pip install --upgrade pip

Note: you may need to restart the kernel to use updated packages.


### __Create a nurse scheduling problem and solve it__

#### 1. Import the libraries

In [33]:
from ortools.sat.python import cp_model

#### 2. Data

Create the data for the example:

In [34]:
num_nurses = 10        #number of nurses - 10
num_shifts = 3        #number of shifts - 3
num_days = 7          #number of days  - 7
all_nurses = range(num_nurses)    #sequence of number of nurses (0, 1, 2, 3, 4, 5, 6, 7, 8 e 9)
all_shifts = range(num_shifts)    #sequence of number of shifts (0, 1 e 2)
all_days = range(num_days)        #sequence of number of days (0, 1, 2, 3, 4, 5 e 6)

In [35]:
shift_requests = [[[0, 1, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0],
                   [0, 0, 0], [0, 0, 0]],
                  [[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 1, 0],
                   [0, 0, 0], [0, 1, 0]],
                  [[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 1, 0], [0, 1, 0],
                   [0, 0, 0], [0, 0, 0]],
                  [[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0],
                   [0, 0, 0], [0, 0, 0]],
                  [[0, 0, 0], [0, 1, 0], [0, 1, 0], [0, 0, 0], [0, 0, 0],
                   [0, 0, 0], [0, 0, 0]],
                  [[0, 0, 0], [0, 1, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0],
                   [0, 0, 0], [0, 0, 0]],
                  [[0, 0, 0], [0, 1, 0], [0, 0, 0], [0, 1, 0], [0, 0, 0],
                   [0, 0, 0], [0, 0, 0]],
                  [[0, 0, 0], [0, 1, 0], [0, 1, 0], [0, 0, 0], [0, 0, 0],
                   [0, 0, 0], [0, 1, 0]],
                  [[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0],
                   [0, 0, 0], [0, 0, 0]],
                  [[1, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 1, 0],
                   [0, 0, 0], [0, 0, 0]]]

#### 3. Create the model

Create the model:

In [36]:
model = cp_model.CpModel()

#### 4. Create the variables

Creates an array of variables:

This array aims to assign shifts to nurses such as:
- shifts [(n, d, s)] equals 1 if shift "s" is assigned to nurse "n" on day "d", and 0 otherwise.

In [37]:
#create boolean variables 'shifts' for each combination of nurse, day and shift
shifts = {}                     #empty dictionary
for n in all_nurses:            #3 nested 'for' loops create the boolean variables and add them to the dictionary
    for d in all_days:
        for s in all_shifts:
            shifts[(n, d, s)] = model.NewBoolVar('shift_n%id%is%i' % (n, d, s))

#### 5. Assign nurses to shifts

Show how to assign nurses to shifts subject to constraints, such as: 
- Max Seq 
- Min Seq
- each shift is assigned to two nurses per day - Condition 1;
- each nurse works at most one shift per day - Condition 2;
- the nurse who works the last shift on one day does not work the morning shift on the next day - Condition 3;-MinRestTime
- the nurse who works the last two consecutive shifts does not work any shift on the next day (no shift) - MaxSeq Condition 4;
- the nurse who works on the last two days (days 5 and 6) have a maximum of 2 shifts on previous days - Condition 5.


Create the first Condition 1:

In [38]:
for d in all_days:
    for s in all_shifts:
        model.Add(sum(shifts[(n, d, s)] for n in all_nurses) == 2)
 #add a constraint that ensures exactly two nurses are assigned to each shift on each day

Create the second Condition 2:

In [39]:
for n in all_nurses:
    for d in all_days:
        model.AddAtMostOne(shifts[(n, d, s)] for s in all_shifts)
        #add a constraint that ensures that each nurse is assigned to at most one shift on each day

#for each nurse, the sum of shifts assigned to that nurse is at most 1 ("at most" because a nurse might have the day off)

Create the third Condition 3:

In [40]:
for n in all_nurses:
    for d in range(num_days - 1):
        last_shift_of_day = shifts[(n, d, num_shifts - 1)]
        first_shift_of_next_day = shifts[(n, d+1, 0)]
        model.Add(last_shift_of_day.Not() + first_shift_of_next_day.Not() >= 1)
#the nurse who works the last shift on one day does not work the morning shift on the next day

Create the fourth Condition 4:

In [41]:
for n in all_nurses:
    for d in range(num_days - 2):
        last_shift_of_day_0 = shifts[(n, d, num_shifts - 1)]
        last_shift_of_day_1 = shifts[(n, d+1, num_shifts - 1)]
        any_shift_of_day_2 = sum(shifts[(n, d+2, s)] for s in range(num_shifts))
        model.Add(last_shift_of_day_0 + last_shift_of_day_1 + any_shift_of_day_2 <= 2)
#the nurse who works the last two consecutive shifts does not work any shift on the next day (no shift)

Create the fifth Condition 5:

In [42]:
nurses_last_two_days = {}

for n in all_nurses:
    nurses_last_two_days[n] = model.NewBoolVar(f"{n}_last_two_days")
    model.Add(shifts[(n, num_days - 1, s)] == 1).OnlyEnforceIf(nurses_last_two_days[n])
    model.Add(shifts[(n, num_days - 2, s)] == 1).OnlyEnforceIf(nurses_last_two_days[n])

for n in all_nurses:
    for d in range(num_days - 2):
        model.Add(sum(shifts[(n, d, s)] for s in all_shifts) <= 2).OnlyEnforceIf(nurses_last_two_days[n])

#add a constraint that ensures that nurses working on the last two days (days 5 and 6) should have a maximum of 2 shifts on previous days

#### 6. Assign shifts evenly

Show how to define assignments for shifts to nurses as evenly as possible. During the seven-day period there are twenty-one shifts, so it is possible to assign four shifts to each of the ten nurses. In this way, there will be two shifts left over, which can be assigned to any nurse.  

In [43]:
min_shifts_per_nurse = ((num_shifts * num_days) * 2) // num_nurses   #minimum number of shifts that each nurse should be assigned based on the total number of shifts, days, and nurses
#shifts to each nurse, but some shifts may be left over. (Here // is the Python integer division operator, which returns the floor of the usual quotient)
#if the total number of shifts and days is not evenly divisible by the number of nurses, then some nurses will be assigned one more shift than others

if ((num_shifts * num_days) * 2) % num_nurses == 0:
    max_shifts_per_nurse = min_shifts_per_nurse
else:
    max_shifts_per_nurse = min_shifts_per_nurse + 1

#assign at least four shifts to each nurse
for n in all_nurses:
    model.Add(sum(shifts[(n, d, s)] for d in all_days for s in all_shifts) >= min_shifts_per_nurse)
    model.Add(sum(shifts[(n, d, s)] for d in all_days for s in all_shifts) <= max_shifts_per_nurse)

#### 7. Objective for the problem

Serves for optimize the following objective function:
- Since shift_requests[n][d][s] * shifts[(n, d, s) is 1 if shift s is assigned to nurse n on day d and that nurse requested that shift (and 0 otherwise), the objective is the number shift of assignments that meet a request.

In [44]:
model.Maximize(
    sum(shift_requests[n][d][s] * shifts[(n, d, s)] for n in all_nurses
        for d in all_days for s in all_shifts))

#### 8. Invoke the solver

Call the solver:

In [45]:
solver = cp_model.CpSolver()
status = solver.Solve(model)

#### 9. Display the results

Displays the following output, which contains an optimal schedule. The output shows which shift assignments were requested and the number of requests that were met:

In [46]:
if status == cp_model.OPTIMAL:
    print('Solution:')
    for d in all_days:
        print('Day', d)
        for n in all_nurses:
            for s in all_shifts:
                if solver.Value(shifts[(n, d, s)]) == 1:
                    if shift_requests[n][d][s] == 1:
                        print('Nurse', n, 'works shift', s, '(requested).')
                    else:
                        print('Nurse', n, 'works shift', s, '(not requested).')
        print()
    print(f'Number of shift requests met = {solver.ObjectiveValue()}', f'(out of {num_nurses * min_shifts_per_nurse})')
else:
    print('No optimal solution found!')


Solution:
Day 0
Nurse 0 works shift 1 (requested).
Nurse 2 works shift 2 (not requested).
Nurse 5 works shift 2 (not requested).
Nurse 6 works shift 0 (not requested).
Nurse 8 works shift 1 (not requested).
Nurse 9 works shift 0 (requested).

Day 1
Nurse 1 works shift 0 (not requested).
Nurse 5 works shift 2 (not requested).
Nurse 6 works shift 1 (requested).
Nurse 7 works shift 1 (requested).
Nurse 8 works shift 0 (not requested).
Nurse 9 works shift 2 (not requested).

Day 2
Nurse 1 works shift 2 (not requested).
Nurse 3 works shift 2 (not requested).
Nurse 4 works shift 1 (requested).
Nurse 6 works shift 0 (not requested).
Nurse 7 works shift 1 (requested).
Nurse 8 works shift 0 (not requested).

Day 3
Nurse 2 works shift 1 (requested).
Nurse 5 works shift 0 (not requested).
Nurse 6 works shift 1 (requested).
Nurse 7 works shift 2 (not requested).
Nurse 8 works shift 0 (not requested).
Nurse 9 works shift 2 (not requested).

Day 4
Nurse 0 works shift 0 (not requested).
Nurse 1 works

#### 10. Statistics

In [32]:
from ortools.sat.python import cp_model

def main():
    # Create a CP-SAT model.
    model = cp_model.CpModel()

    # Define the number of nurses, shifts, and days
    num_nurses = 10
    num_shifts = 3
    num_days = 7

    # Initialize the shifts dictionary
    shifts = {}
    shift_requests = [[[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0],
                   [0, 0, 0], [0, 0, 0]],
                  [[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 1, 0],
                   [0, 0, 0], [0, 0, 0]],
                  [[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0],
                   [0, 0, 0], [0, 0, 0]],
                  [[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0],
                   [0, 0, 0], [0, 0, 0]],
                  [[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0],
                   [0, 0, 0], [0, 0, 0]],
                  [[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0],
                   [0, 0, 0], [0, 0, 0]],
                  [[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0],
                   [0, 0, 0], [0, 0, 0]],
                  [[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0],
                   [0, 0, 0], [0, 0, 0]],
                  [[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0],
                   [0, 0, 0], [0, 0, 0]],
                  [[1, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0],
                   [0, 0, 0], [0, 0, 0]]]

    # Create boolean variables for each nurse, day, and shift combination
    for n in range(num_nurses):
        for d in range(num_days):
            for s in range(num_shifts):
                shifts[(n, d, s)] = model.NewBoolVar('shift_n%id%is%i' % (n, d, s))

    # Constraints
    

    # 2. MaxSeq: Maximum number of consecutive shifts for a nurse
    max_seq = 5  # Example: maximum consecutive shifts allowed
    for n in range(num_nurses):
        for d in range(num_days - max_seq + 1):
            for s in range(num_shifts):
                # Ensure no more than 'max_seq' consecutive shifts
                model.Add(sum(shifts[(n, d + i, s)] for i in range(max_seq)) <= max_seq)
                
    # 3. MinSeq: Minimum number of consecutive shifts for a nurse
    min_seq = 2  # Example: minimum consecutive shifts required
    for n in all_nurses:
        for d in range(num_days - min_seq + 1):
            for s in all_shifts:
                # Ensure at least 'min_seq' consecutive shifts
                model.Add(sum(shifts[(n, d + i, s)] for i in range(min_seq)) >= min_seq)

    model.Maximize(
        sum(shift_requests[n][d][s] * shifts[(n, d, s)] for n in all_nurses
            for d in all_days for s in all_shifts))

    # Create a solver and solve the model.
    solver = cp_model.CpSolver()
    status = solver.Solve(model)

    # Output the results.
    if status == cp_model.OPTIMAL:
        print('Solution:')
        for d in range(num_days):
            print('Day', d)
            for n in range(num_nurses):
                for s in range(num_shifts):
                    if shift_requests[n][d][s] == 1:
                        print('Nurse', n, 'works shift', s, '(requested).')
                    else:
                        print('Nurse', n, 'works shift', s, '(not requested).')
            print()
        print(f'Number of shift requests met = {solver.ObjectiveValue()}', f'(out of {num_nurses * min_shifts_per_nurse})')          
    else:
        print('No optimal solution found!')

if __name__ == '__main__':
    main()


Solution:
Day 0
Nurse 0 works shift 0 (not requested).
Nurse 0 works shift 1 (not requested).
Nurse 0 works shift 2 (not requested).
Nurse 1 works shift 0 (not requested).
Nurse 1 works shift 1 (not requested).
Nurse 1 works shift 2 (not requested).
Nurse 2 works shift 0 (not requested).
Nurse 2 works shift 1 (not requested).
Nurse 2 works shift 2 (not requested).
Nurse 3 works shift 0 (not requested).
Nurse 3 works shift 1 (not requested).
Nurse 3 works shift 2 (not requested).
Nurse 4 works shift 0 (not requested).
Nurse 4 works shift 1 (not requested).
Nurse 4 works shift 2 (not requested).
Nurse 5 works shift 0 (not requested).
Nurse 5 works shift 1 (not requested).
Nurse 5 works shift 2 (not requested).
Nurse 6 works shift 0 (not requested).
Nurse 6 works shift 1 (not requested).
Nurse 6 works shift 2 (not requested).
Nurse 7 works shift 0 (not requested).
Nurse 7 works shift 1 (not requested).
Nurse 7 works shift 2 (not requested).
Nurse 8 works shift 0 (not requested).
Nurse 8 w

No optimal solution found!


In [47]:
print('\nStatistics')
print('  - conflicts: %i' % solver.NumConflicts())
print('  - branches : %i' % solver.NumBranches())
print('  - wall time: %f s' % solver.WallTime())


Statistics
  - conflicts: 0
  - branches : 420
  - wall time: 0.060094 s
